Compare specific energy use across buildings, looking at different features.

In [177]:
import pandas as pd
import plotly.express as px
import os

In [178]:
metadata = pd.read_excel("../data/raw/Potential objects.xlsx")
metadata.columns

Index(['Building', 'Address', 'Use Type', 'Addr meter', 'Energy', 'Unit',
       'Heat', 'measurement interval', 'Note', 'Data for DKSR',
       'Energy source', 'Heating output (kW)', 'Heating output - numeric',
       'Energy consumption (m3, GJ)', 'School hours, holiday calendar',
       'Variable or fixed gas pricing (Kč without DPH)',
       'Energy reference surface (m2)', 'Volume (m3)', 'EPC', 'EPC - numeric',
       'Number of floors', 'Floors above ground', 'Floors below ground',
       'Number of classrooms', 'Number of gyms', 'Number of kitchen',
       'Building height (m)', 'Year of construction', 'Materials',
       'Heat insulation', 'Windows', 'Windows area (m2)'],
      dtype='object')

In [179]:
metadata[['Heating output - numeric']]

,Heating output - numeric
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [180]:
uses = pd.DataFrame(metadata["Use Type"].value_counts())
fig = px.bar(uses, 
             #names=uses.index, 
             #values=uses["count"], 
             width=600, height=400,
             title="Building Use Types",
             template="plotly_white",
             text_auto=True,
             orientation="h",
             barmode="stack")
fig.update_layout(showlegend=False,
                  xaxis_title="Count",
                  yaxis_title="",)

fig.write_image("../reports/building_types.png")
fig.show()

In [181]:
all_buildings_yearly_kwh = pd.read_parquet("../data/processed/all_buildings_yearly_kwh.parquet")

merge energy demand into metadata

In [182]:
demand_yearly_kwh = all_buildings_yearly_kwh.T
demand_yearly_kwh.rename(columns={c:f"{c}_kWh" for c in demand_yearly_kwh.columns}, inplace=True)
demand_yearly_kwh.fillna(0, inplace=True)
demand_yearly_kwh
## Delete 10.10PF1 - invalid data for 2022

,2021_kWh,2022_kWh,2023_kWh
10.10-PF1,0.000,57384.004,1.476948e+05
10.25-PF1,797269.782,716797.432,6.198221e+05
10.26-TF1,0.000,0.000,5.305042e+05
10.28-PF1,756792.992,639437.036,6.573794e+05
10.33-PF1,683271.742,591185.708,5.364752e+05
10.4-TF1,0.000,0.000,4.445869e+05
10.6-PF2,283316.142,547596.964,3.167349e+05
10.9.2-PF1,0.000,0.000,1.455689e+05
2.2-PF1,0.000,376257.696,3.694242e+05
2.3.-PF1,886222.474,997653.954,8.261008e+05


In [183]:
def get_demand(row, col):
    try:
        return demand_yearly_kwh.loc[row["Addr meter"].replace("/", "-"), col] 
    except: 
        return None

for c in demand_yearly_kwh.columns:
    metadata[c] = metadata.apply(lambda row: get_demand(row, c), axis=1)


In [184]:
metadata["Energy reference surface (m2)"].notna().sum() 

11

In [185]:
metadata["23-22-21"] = metadata["2021_kWh"] - metadata["2022_kWh"] - metadata["2021_kWh"]
metadata.sort_values("23-22-21", inplace=True)
metadata["Year of construction"] = metadata["Year of construction"].astype(str).str.replace(".0", "").replace("nan", "-")
metadata["plot_name"] = metadata["Building"] + "<br>" + metadata["Use Type"] + "<br>" + metadata["Year of construction"]
metadata = metadata.dropna(subset=demand_yearly_kwh.columns, how="all").reset_index()
annual_energy = metadata.loc[:, list(demand_yearly_kwh.columns) + ["plot_name"]].set_index("plot_name")
annual_energy


,2021_kWh,2022_kWh,2023_kWh
plot_name,,,
DS Chodov<br>Social (Elderly home)<br>1990,1725412.736,1345743.654,974415.8920
DS Elišky Purkyňové<br>Social<br>2012,0.000,780202.158,671037.8060
ZŠ Karla Čapka<br>School<br>1915,797269.782,716797.432,619822.1012
ZŠ Gutova<br>School<br>1970,756792.992,639437.036,657379.4292
ZŠ U Vršovického nádraží<br>School<br>1930,683271.742,591185.708,536475.2040
Areál Gutova<br>Sport areal<br>-,283316.142,547596.964,316734.8920
Gymnázium Na Vítězné pláni<br>School<br>1961,0.000,376257.696,369424.2300
MŠ Kodaňská<br>School<br>-,0.000,57384.004,147694.8340
ZŠ Brigádníků<br>School<br>1960,0.000,0.000,530504.2440


In [145]:
fig = px.bar(metadata.loc[:, list(demand_yearly_kwh.columns) + ["plot_name"]].set_index("plot_name"), 
       barmode="group",
       title="Annual Energy Demand (kWh)",
       template="plotly_white",
       #orientation="h",
       width=1000,
       height=600)

fig.update_layout(yaxis_title="kWh",
                  xaxis_title="",
                  legend_title="Sensor")
fig.write_image("../reports/annual_energy_demand.png")
fig.show()

# Adjust for weather

In [204]:
weather_hourly = pd.read_csv("../data/interim/temperature_features.csv", index_col="Unnamed: 0").iloc[:, -2:]
weather_hourly.index = pd.to_datetime(weather_hourly.index)
weather_hourly = weather_hourly.loc[(weather_hourly.index>"01.01.2021") & (weather_hourly.index<"31.12.2023")]
hdd_annual = weather_hourly.resample("Y").sum()
hdd_annual.index = hdd_annual.index.year
fig = px.bar(hdd_annual["hdh_t_mean<15"], 
       barmode="group",
       template="plotly_white",
       title="Heating Degree Hours below 15°C",
       width=500,
       height=400)
fig.update_layout(
    showlegend=False,
    yaxis_title="Heating Degree Hours",
    xaxis_title=""
)
fig.show()
fig.write_image("../reports/heating_degree_hours.png")

In [198]:
hdd_annual["hdh_t_mean<15"]

2021    62491.316667
2022    53729.225000
2023    50062.925000
Name: hdh_t_mean<15, dtype: float64

In [ ]:
annual_energy_weather_adjusted = a

Specific Demand

In [146]:
for c in demand_yearly_kwh.columns:
    metadata[f"{c}/m²"] = metadata[c] / metadata["Energy reference surface (m2)"]

In [157]:
specific_demand_cols = [c for c in metadata.columns if c.endswith("kWh/m²")]
temp_df = metadata.loc[:, specific_demand_cols + ["Building", "EPC - numeric", "plot_name"]].set_index("Building")

temp_df.rename(columns={"EPC - numeric": "EPC kWh/m²"}, inplace=True)
temp_df["valid"] = temp_df.drop("plot_name", axis=1).sum(axis=1) - temp_df["EPC kWh/m²"]

temp_df = temp_df.loc[temp_df["valid"]>0, :]
temp_df.drop("valid", axis=1, inplace=True)
temp_df.set_index("plot_name", inplace=True)

fig = px.bar(temp_df,
       barmode="group",
       template="plotly_white",
       title="Annual Primary Energy Demand (kWh/m²)<br>Measured vs. Label",
       width=800,
       height=400)
fig.update_layout(
       xaxis_title = "Building",
       yaxis_title = "kWh/m²"   
)
fig.show()
fig.write_image("../reports/annual_primary_energy_demand.png")

,2021_kWh/m²,2022_kWh/m²,2023_kWh/m²,EPC kWh/m²
plot_name,,,,
DS Chodov<br>Social (Elderly home)<br>1990,153.889827,120.027083,86.908303,224.7
DS Elišky Purkyňové<br>Social<br>2012,0.000000,106.862369,91.910397,181.2
ZŠ U Vršovického nádraží<br>School<br>1930,118.029321,102.122250,92.671481,207.5
Gymnázium Na Vítězné pláni<br>School<br>1961,0.000000,60.404189,59.307149,96.1
ZŠ Brigádníků<br>School<br>1960,0.000000,0.000000,71.189512,243.9
ZŠ Švehlova<br>School<br>1970,0.000000,0.000000,50.463892,105.0
